<a href="https://colab.research.google.com/github/sanmish4ds/cuckoo/blob/master/incident_category.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import files 
uploaded = files.upload() 

Saving ar_incident_old.csv to ar_incident_old (1).csv


In [0]:
import io 
import pandas as pd
df_a = pd.read_csv('/content/sample_data/active_incident.csv',encoding='latin1')
df_b = pd.read_csv('/content/sample_data/archive_incident.csv',encoding='latin1')
df_c = pd.read_csv(io.BytesIO(uploaded['ar_incident_old.csv']),encoding='latin1')
##df_c = pd.read_csv('/content/sample_data/ar_incident_old.csv',encoding='latin1')# Dataset is now stored in a Pandas Dataframe 

In [0]:
df_c = df_c.drop(axis =1 ,columns='sys_archived')

In [0]:
#display(df_a.head())
#display(df_b.head())
#display(df_c.head())
frames = [df_a,df_b,df_c]
df = pd.concat(frames)
df.shape

(24125, 16)

In [0]:
df_x= df[['short_description','cmdb_ci']]
df_x = df_x.dropna(axis=0)
df_x

,short_description,cmdb_ci
0,Tronweb-Multiple errors for the new fields add...,TronWeb - Reports
1,01046475619 (This is impacting quotes in all L...,AFE
2,Fireline Score Correction for Zip Code Level S...,TronWeb - Issue
3,Good Driver discount given for a suspended lic...,TronWeb - Issue
4,Tronweb Renewal - incomplete print produced for,TronWeb - Print
...,...,...
9995,Submit proof of insurance to the DMV for polic...,TronWeb - Interface
9996,CURRENT CARRIER - AUTOMATED REJECT REPORT FOR ...,TronWeb - Interface
9997,***Job FTPCOMEF52 ended abnormally. 09/17/17 2...,AS400A
9998,***Job GC01670 ended abnormally. 05/04/18 05:0...,AS400A


In [0]:
df_x['category_id'] = df_x['cmdb_ci'].factorize()[0]

In [0]:
df_x.groupby('cmdb_ci').cmdb_ci.count()

cmdb_ci
ACCORD Products                      1
ACIC Agency Port                     1
ACIC V12 - Issue                     2
ACIC Voice Response Unit - VRU       2
AFE                               3432
                                  ... 
Website External - Generic           1
adms                                 2
refresh                              2
webMethods                          17
www.mapfreconnect.com                2
Name: cmdb_ci, Length: 93, dtype: int64

In [0]:
#df_x = df_x.drop_duplicates()
category_id_df = df_x[['cmdb_ci', 'category_id']].drop_duplicates().sort_values('category_id')
category_to_id = dict(category_id_df.values)
id_to_category = dict(category_id_df[['category_id', 'cmdb_ci']].values)

###tokenize the text


In [0]:
#df_x.shape
import matplotlib.pyplot as plt
fig = plt.figure(figsize=(50,30))
df_x.groupby('cmdb_ci').short_description.count().plot.bar(ylim=0)
plt.show()

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
tfidf = TfidfVectorizer(sublinear_tf=True, min_df=5, norm='l2', encoding='latin-1', ngram_range=(1, 2), stop_words='english')
features = tfidf.fit_transform(df_x.short_description).toarray()
labels = df_x.category_id
features.shape

(23704, 6432)

In [0]:
from sklearn.feature_selection import chi2
import numpy as np
N = 2
for Product, category_id in sorted(category_to_id.items()):
  features_chi2 = chi2(features, labels == category_id)
  indices = np.argsort(features_chi2[0])
  feature_names = np.array(tfidf.get_feature_names())[indices]
  unigrams = [v for v in feature_names if len(v.split(' ')) == 1]
  bigrams = [v for v in feature_names if len(v.split(' ')) == 2]
  print("# '{}':".format(Product))
  print("  . Most correlated unigrams:\n. {}".format('\n. '.join(unigrams[-N:])))
  print("  . Most correlated bigrams:\n. {}".format('\n. '.join(bigrams[-N:])))

# 'ACCORD Products':
  . Most correlated unigrams:
. locks
. accord
  . Most correlated bigrams:
. dmv policy
. proof insurance
# 'ACIC Agency Port':
  . Most correlated unigrams:
. errored
. dr
  . Most correlated bigrams:
. policy documents
. processed endorsement
# 'ACIC V12 - Issue':
  . Most correlated unigrams:
. access
. v12
  . Most correlated bigrams:
. company 80
. need access
# 'ACIC Voice Response Unit - VRU':
  . Most correlated unigrams:
. issue
. vru
  . Most correlated bigrams:
. dmv policy
. proof insurance
# 'AFE':
  . Most correlated unigrams:
. problem
. afe
  . Most correlated bigrams:
. problem afe
. new problem
# 'AFEINT01_appil001-grcella':
  . Most correlated unigrams:
. errors
. login
  . Most correlated bigrams:
. dmv policy
. proof insurance
# 'AIXPRD1':
  . Most correlated unigrams:
. mailbox_rp
. aixprd1
  . Most correlated bigrams:
. dmv policy
. proof insurance
# 'APPPL001-GR':
  . Most correlated unigrams:
. log
. packet
  . Most correlated bigrams:
. p

In [0]:
#from sklearn.model_selection import train_test_split
#from sklearn.feature_extraction.text import CountVectorizer
#from sklearn.feature_extraction.text import TfidfTransformer
#from sklearn.naive_bayes import MultinomialNB
#X_train, X_test, y_train, y_test = train_test_split(df_x['short_description'], df_x['cmdb_ci'], random_state = 0)
#count_vect = CountVectorizer()
#X_train_counts = count_vect.fit_transform(X_train)
#tfidf_transformer = TfidfTransformer()
#X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
#clf = MultinomialNB().fit(X_train_tfidf, y_train)##


In [0]:
#print(clf.predict(count_vect.transform(["(Reopen INC1743263) - 8001010072811_Driver Class Ranking error due to QUO id"])))

['TronWeb - Issue']


In [0]:
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
X_train_svc,X_test_svc,y_train_svc,y_test_svc = train_test_split(features, labels,test_size=0.30,random_state=42)
clf1 = LinearSVC().fit(X_train_svc, y_train_svc)



In [0]:
clf1.score(X_train_svc,y_train_svc)

0.9316538090646095

In [0]:
X_test_svc

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [0]:

input_text = "cancellation"
c=clf1.predict(tfidf.transform([input_text]))
category_id_df = category_id_df[category_id_df.values == c] 
category_id_df

,cmdb_ci,category_id
4,TronWeb - Print,3
